# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


## Again - please note: 2 important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Kernel menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.28.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00


In [10]:
!pip install -q google-generativeai google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.5 MB/s eta 0:00:00


In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

Mounted at /content/drive


# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [5]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [7]:

# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [26]:
from google.oauth2 import service_account
key_path="/content/drive/MyDrive/llms/gen-lang-client-0290724549-888ce2d4e671.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

In [44]:
from google.cloud import speech
from google.cloud import storage
import google.generativeai as genai

# Set up Gemini API key
google_api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=google_api_key)
model = genai.GenerativeModel('gemini-2.0-flash-exp')

# Initialize the Speech-to-Text client
client = speech.SpeechClient(credentials=credentials)

# Initialize the Storage client
storage_client = storage.Client(credentials=credentials)

In [35]:
# Upload the audio file to GCS first
bucket_name = "llmengineering"  # Replace with your bucket name
destination_blob_name = "audio_files/denver_extract.mp3"  # Choose a path in your bucket
source_file_name = audio_filename  # Your local file path

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

print(f"File {source_file_name} uploaded to gs://{bucket_name}/{destination_blob_name}")

File /content/drive/MyDrive/llms/denver_extract.mp3 uploaded to gs://llmengineering/audio_files/denver_extract.mp3


In [36]:
# Create the audio object from the GCS URI
gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"
audio = speech.RecognitionAudio(uri=gcs_uri)

In [39]:
# Configure audio and recognition settings
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.MP3,
    sample_rate_hertz=16000,  # adjust this to match your audio file specs
    language_code="en-US",
)

In [41]:
operation = client.long_running_recognize(config=config, audio=audio)
print("Waiting for operation to complete...")
response = operation.result(timeout=600)

# Collect the full transcription
transcription = " ".join([result.alternatives[0].transcript for result in response.results])

# Print the transcription
print("Transcription:\n", transcription)

Waiting for operation to complete...
Transcription:
 kind of the Confluence of this whole idea of the Confluence week the merging of two rivers and has we've kind of seen recently in politics and in the world there's a lot of  situations where water is very important right now and some very big issues so that is the reason the back of the logo is considered water so  see  the creation of the logo here  so that basically kind of sums of the reason behind the logo and the all the meanings behind the symbolism and you'll hear a little bit more about our Confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of indigenous people say thank you thank you so much and thanks for your leadership  all right  welcome to the Denver city council meeting of Monday October 9th police rides with the Pledge of Allegiance by councilman Lopez  I pledge allegian

In [46]:
# Send to Gemini model for processing (e.g., summarization)
gemini_response = model.generate_content(f"Summarize this transcription:\n{transcription}")
print("\nGemini Summary:\n", gemini_response.text)



Gemini Summary:
 This transcription is of a Denver City Council meeting on October 9th, 2017. Here's a summary:

*   **Confluence Week & Logo Explanation:** A speaker describes the symbolism behind the "Confluence Week" logo, connecting it to the merging of rivers, the importance of water issues, and Denver's indigenous heritage. The week highlights indigenous events in Denver to promote unity.
*   **Pledge of Allegiance & Roll Call:** The meeting begins with the Pledge of Allegiance and a roll call to establish a quorum.
*   **Halloween Parade Announcement:** Councilman Clark announces a Halloween parade.
*   **Indigenous Peoples' Day Proclamation:** A proclamation (#1127) is read to observe the second annual Indigenous Peoples' Day in Denver. It acknowledges the history of indigenous people on the land, Denver's origins at the confluence of rivers that were historically indigenous homelands, and recognizes the contributions of indigenous culture to Denver.
*   **Proclamation Adoptio

In [47]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [48]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [49]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
kind of the Confluence of this whole idea of the Confluence week the merging of two rivers and has we've kind of seen recently in politics and in the world there's a lot of  situations where water is very important right now and some very big issues so that is the reason the back of the logo is considered water so  see  the creation of the logo here  so that basically kind of sums of the reason behind the logo and the all the meanings behind the sym

In [50]:
response = tokenizer.decode(outputs[0])

In [51]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
kind of the Confluence of this whole idea of the Confluence week the merging of two rivers and has we've kind of seen recently in politics and in the world there's a lot of  situations where water is very important right now and some very big issues so that is the reason the back of the logo is considered water so  see  the creation of the logo here  so that basically kind of sums of the reason behind the logo and the all the meanings behind the symbolism and you'll hear a little bit more about our Confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of indigenous people say thank you thank you so much and thanks for your leadership  all right  welcome to the Denver city council meeting of Monday October 9th police rides with the Pledge of Allegiance by councilman Lopez  I pledge allegiance to the flag of the United States of America to The Republic  all right thank you councilman Lopez Madam Secretary Rock Hall  black  Clark Espinosa  Gilmore here  Cashman  Lopez new Ortega president have a quorum through all the minutes are there any corrections to the minutes of October 2nd seen none minutes of October 2nd and approved Council announcements are there any announcements by members of council  Calvin Clark I just wanted to invite everyone down to the first-ever Halloween parade on Broadway and lucky district 7 it will happen on Saturday October 21st at 6 p.m. and will move along Broadway from third to Alameda it's going to be a fun family-friendly event everyone's invited to come down where costume there'll be candy for the kids and there are Kiki zombies and 29 horses and all kinds of fun and funky stuff on the fun and funky part of Broadway so please join us October 21st at 6 for the Broadway Halloween parade thing this President alright thank you councilman Clark I will be there all right presentation Mountain Dew have any presentations. We do have one Proclamation this evening Proclamation 1127 and abs  Prince of the annual indigenous people's day in the City and County of Denver Council Lopez will you please read it and number 17 number 1127 series of 2017 in observance of the second annual indigenous people's day in the City and County of Denver where is the council the City and County of Denver recognizes that the indigenous peoples have lived and flourished on the last known as the America since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes which include the southern Ute the Ute Mountain Ute tribe of Colorado and whereas the tribal Homeland and seasonal encampments of the Arapaho and Cheyenne people along the banks are there Cherry Creek and South Platte River Confluence gave bearing to the Future settlements that would become the birthplace of the mile high city  and whereas Colorado and Compasses the homelands of 48 tribes in the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal Nations  and whereas on October 3rd 2016 the City and County of Denver unanimously passed Council Bill 801 series of 2016 officially designating the second Monday of October of each year as indigenous people stay in Denver Colorado  and whereas the council the City and County of Denver continues to recognize and value the vast contributions made to community made to the community Through indigenous people's knowledge science philosophy arts and culture and through these contributions the city of Denver has developed in thrive  brass  the indigenous Community especially you have made great efforts this year to draw attention to the contributions of indigenous people including Confluence week driving record attendance to a national indigenous youth leadership conference leading conversations on inclusion with their peers and supporting increase indigenous youth participation in science and engineering now therefore be a clean by this Council of City and County of Denver section one at the Council of the City and County of Denver celebration honors the cultural and foundational contributions of indigenous people to our history  past present and future and continues to promote the education of the Denver the Denver Community about these historic and contemporary contributions of indigenous people  section 2 at the City and County of Denver Colorado does hereby observed October 9th 2017 as indigenous people stay Section 3 at the clerk of the City and County of Denver show a test and fix the Seal of the City and County of Denver to this Proclamation and that a copy of you transmit trans made it gives me to the Denver American Indian commission the City and County of Denver School District number one in the Colorado Commission on Indian Affairs  thank you councilman Lopez your motion to adopt president of 2017 be adopted all right it has been moved and 2nd and council members of council Council Lopez thank you mr. president it gives me a lot of pleasure and pride to read this Proclamation officially for this for the third time but as indigenous people's Day in Denver fish me for the second time it is it's always awesome to be able to see not just this Proclamation come through come by my desk to see so many different people and it from our community in our in our council chambers it was very beautiful piece of artwork that you presented to us earlier and it is exactly the spirit that we drafted this Proclamation and this actual that the ordinance that created indigenous people's day when we sat down and wrote it and it has a community  we couldn't think of anything else to begin except for the Confluence of the two rivers and those compliments of the two rivers created such a great City and we live in such an amazing City we are all proud of it and sometimes we in a lot of people from all over the country are for all over the world a proud of it and sometimes a little too proud of his tell him to go back home, I'm kidding when I say that but the really nice thing about this is that we are celebrating an indigenous people's day out of Pride truck we are who we are as a city in the in the contributions of indigenous people to the city not out of spite not out of a replacement of one culture over the other or or out of contempt or or disrespect you know I think of like a quote that says or Travis made very very popular  and I get stuck with me for a very long time and I'm now a night in anywhere anytime I have the opportunity to speak in front of children and especially children in our community that you know they  often second-guess themselves and where they're coming from who they are and and I always say that you know it's it's very important to be proud of who we are from and the quote that I use some sister Travis's you no pride in one's own culture does not require contempt or disrespect of another right and that's very important it's a very important for us to recognize that no matter who we are where we come from in this society that your pride in your own culture doesn't require to not require the contempt or disrespect of another a man what a year to be for that to just sit on our shoulders for a while for us to think about right and so I wanted to thank you all the commission there's going to be a couple individuals that are going to come speak thank you for your art your lovely artwork for us to see what's in your heart and what now is become Bobby's going to be a very important symbol  for the community and also just for the work is the daily work every single day we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now I'll stand on street corners right and poverty without access to Services right without access to sobriety or even housing or jobs and what a what  what a cruel way to pay back a culture that has paved the way for the city to be built upon its Shores right so you have a lot of work to do and these kind of proclamation in this day is not a day off for today on in Denver right and in in addressing those those those critical issue so I know that my colleagues are very supportive I'm going to ask you to support this Proclamation as I know you always have done in the past I'm very proud of today I'm celebrating indigenous people's day I wanted to make a point out of that  thank you councilman Lopez and thank you for sponsoring this Council marteka my name be added I don't think I could add much more to what councilman Lopez has shared with us I want to thank him for bringing this forward and really just appreciate all the contributions that are Native American Community has contributed to this great City and great state I worked in the lieutenant governor's office when the commission on Indian Affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the Utes and the Comanches that had been short of it at odds with each other for about a hundred years and just being able to participate in that how I was was pretty awesome so in for those of you who continue to participate in the annual pow-wow it's it's such a great  opportunity for everybody else to enjoy so many of the contributions of the culture went to see that the dance continues to be carried on as well as is the native language from generation to generation is just so incredible because in so many cultures you know people have come here and assimilated to the you know the Norms here and they lose their language and and and lose a lot of the culture and in the native community that that hasn't happened that has that you don't  commitment to just passing that on from generation to generation is is so important and so I'm I'm happy to be a co-sponsor of this tonight thank you  alright thank you councilman Ortega Council when we can each thank you very much and I also want to thank my colleagues for bringing this forward and I I just wanted to to stay where it's at artist about how beautiful and moving and I thought this logo was and your description of it and I think one of the things that is clear as you know the words sometimes don't convey the power of imagery or music or the other pieces that make up culture and so I think the art is so important and you talked about water I was also thinking about land and I guess I just wanted to say thank you many of the Native American peoples of Colorado I've been at the Forefront actually nationally of Defending some of the the public lands that have been protected over the last few years that are under attack right now and they're places that you all the communities have fought to protect but that everyone gets to enjoy and so I just think that it's an example of where cultural preservation intersects with Environmental Protection within a  Ken and all of the other ways that that public lands are so important and so I think I'm I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now and so as we celebrate I appreciate that there's still a piece of resistance in here and I think that I just want to mention a solidarity and a mention a feeling of solidarity with that resistance so thank you and happy Confluence week thank you councilman can each and seeing other comments I'll just say a couple and in a time of such divisive ugliness and just despicable behavior from our leadership the reason I'm still support of indigenous people day is because it means inclusivity it means respecting all respecting those who have been silenced on purpose for a long time and whose history is now  I told and so we celebrate inclusivity in the face of such such evil x on it<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of Denver City Council Meeting**
==========================================

**Summary**
------------

The Denver City Council met on Monday, October 9th to discuss the proclamation of Indigenous Peoples' Day. The council recognized the contributions of indigenous people to the city's history, culture, and development. They also acknowledged the importance of preserving the cultural heritage of Native American communities.

**Attendees**
------------

*   Madam Secretary Rock Hall
*   Councilman Clark
*   Councilman Lopez
*   Councilman Ortega
*   Councilman Espinosa
*   Councilman Gilmore
*   Councilman Cashman

**Location and Date**
---------------------

The meeting took place at the Denver City Council chambers on Monday, October 9th.

**Discussion Points**
--------------------

*   The proclamation of Indigenous Peoples' Day to recognize the contributions of indigenous people to the city's history, culture, and development.
*   The importance of preserving the cultural heritage of Native American communities.
*   The significance of the Confluence of the two rivers, which represents the merging of two cultures.

**Takeaways**
-------------

*   The city of Denver recognizes the importance of indigenous peoples' contributions to its history and culture.
*   The council acknowledges the need to preserve the cultural heritage of Native American communities.
*   The proclamation of Indigenous Peoples' Day promotes inclusivity and respect for all cultures.

**Action Items**
----------------

1.  **Proclamation of Indigenous Peoples' Day**: The council adopted Proclamation 1127, series of 2017, to observe October 9th, 2017, as Indigenous Peoples' Day in the City and County of Denver.
2.  **Community Engagement**: The council encourages community members to participate in the celebration of Indigenous Peoples' Day and to learn about the cultural heritage of Native American communities.
3.  **Preservation of Cultural Heritage**: The council recognizes the importance of preserving the cultural heritage of Native American communities and encourages efforts to protect and promote their traditions.

**Owners**
----------

*   **Councilman Lopez**: Sponsor of the proclamation and advocate for the recognition of indigenous peoples' contributions.
*   **Councilman Ortega**: Co-sponsor of the proclamation and supporter of the preservation of cultural heritage.
*   **Councilman Clark**: Organizer of the Halloween parade and supporter of the proclamation.
*   **Councilman Cashman**: Supporter of the proclamation and advocate for community engagement.

**Next Steps**
----------------

The council will continue to promote the celebration of Indigenous Peoples' Day and encourage community members to participate in the preservation of cultural heritage.<|eot_id|>

# Student contribution

Student Emad S. has made this powerful variation that uses `TextIteratorStreamer` to stream back results into a Gradio UI, and takes advantage of background threads for performance! I'm sharing it here if you'd like to take a look at some very interesting work. Thank you, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D

## Alternative implementation

Class student Youssef has contributed this variation in which we use an open-source model to transcribe the meeting Audio.

Thank you Youssef!

In [1]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

NameError: name 'AutoModelForSpeechSeq2Seq' is not defined

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename)

In [ ]:
transcription = result["text"]
print(transcription)